We will begin by importing the relevant packages i.e., Pandas, Sklearn Packages

In [437]:
%matplotlib inline

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

We will then import the credit risk dataset using pandas

In [438]:
original_dataset = pd.read_csv('credit_risk_dataset.csv')
original_dataset

,Unnamed: 0,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,30786,41,40000,RENT,3.0,PERSONAL,A,9200,7.49,0,0.23,N,13
1,29460,44,28000,OWN,0.0,VENTURE,A,3500,8.94,0,0.13,N,12
2,7059,22,56000,RENT,0.0,DEBTCONSOLIDATION,B,7000,11.36,0,0.13,N,2
3,5377,24,45000,MORTGAGE,2.0,PERSONAL,A,7200,7.29,0,0.16,N,4
4,27170,28,55000,RENT,3.0,HOMEIMPROVEMENT,E,15000,17.06,0,0.27,Y,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24430,9887,23,45000,RENT,7.0,EDUCATION,B,9000,10.65,0,0.20,N,2
24431,15196,26,125000,MORTGAGE,3.0,EDUCATION,D,24000,15.28,0,0.19,Y,4
24432,32218,43,138000,MORTGAGE,4.0,EDUCATION,A,9900,6.99,0,0.07,N,14
24433,25398,27,84780,MORTGAGE,5.0,VENTURE,A,18000,7.51,0,0.21,N,10


There are some empty cells on the person_emp_length and loan_int_rate.
Perform data cleaning to replacing empty fields with the mean of the columns

In [439]:
dataset = original_dataset.copy()
person_emp_length_mean = dataset["person_emp_length"].mean()
loan_int_rate_mean = dataset["loan_int_rate"].mean()

print("'person_emp_length' mean:" + str(person_emp_length_mean) + ", 'loan_int_rate' mean:" + str(loan_int_rate_mean))

dataset["person_emp_length"].fillna(person_emp_length_mean,inplace = True)
dataset["loan_int_rate"].fillna(loan_int_rate_mean,inplace = True)
dataset

'person_emp_length' mean:4.778072167549836, 'loan_int_rate' mean:11.006376791932013


,Unnamed: 0,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,30786,41,40000,RENT,3.0,PERSONAL,A,9200,7.49,0,0.23,N,13
1,29460,44,28000,OWN,0.0,VENTURE,A,3500,8.94,0,0.13,N,12
2,7059,22,56000,RENT,0.0,DEBTCONSOLIDATION,B,7000,11.36,0,0.13,N,2
3,5377,24,45000,MORTGAGE,2.0,PERSONAL,A,7200,7.29,0,0.16,N,4
4,27170,28,55000,RENT,3.0,HOMEIMPROVEMENT,E,15000,17.06,0,0.27,Y,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24430,9887,23,45000,RENT,7.0,EDUCATION,B,9000,10.65,0,0.20,N,2
24431,15196,26,125000,MORTGAGE,3.0,EDUCATION,D,24000,15.28,0,0.19,Y,4
24432,32218,43,138000,MORTGAGE,4.0,EDUCATION,A,9900,6.99,0,0.07,N,14
24433,25398,27,84780,MORTGAGE,5.0,VENTURE,A,18000,7.51,0,0.21,N,10


There exists several fields with categorical values. We are going to encoding of categorical values.
We need to map the fields as follows:
1. "cb_person_default_on_file" field to where Y=1 and N=0
2. "person_home_ownership" field to where RENT=1, OWN=2, MORTGAGE=3, OTHER=4
3. "loan_intent" field to where PERSONAL=1, VENTURE=2, DEBTCONSOLIDATION=3, HOMEIMPROVEMENT=4, EDUCATION=5, MEDICAL=6
4. "loan_grade" field to where A=1, B=2, C=3, D=4, E=5, F=6, G=7

In [440]:
dataset["cb_person_default_on_file"] = dataset["cb_person_default_on_file"].replace(["Y"], 1)
dataset["cb_person_default_on_file"] = dataset["cb_person_default_on_file"].replace(["N"], 0)
dataset["person_home_ownership"] = dataset["person_home_ownership"].replace(["RENT"], 1)
dataset["person_home_ownership"] = dataset["person_home_ownership"].replace(["OWN"], 2)
dataset["person_home_ownership"] = dataset["person_home_ownership"].replace(["MORTGAGE"], 3)
dataset["person_home_ownership"] = dataset["person_home_ownership"].replace(["OTHER"], 4)
dataset["loan_intent"] = dataset["loan_intent"].replace(["PERSONAL"], 1)
dataset["loan_intent"] = dataset["loan_intent"].replace(["VENTURE"], 2)
dataset["loan_intent"] = dataset["loan_intent"].replace(["DEBTCONSOLIDATION"], 3)
dataset["loan_intent"] = dataset["loan_intent"].replace(["HOMEIMPROVEMENT"], 4)
dataset["loan_intent"] = dataset["loan_intent"].replace(["EDUCATION"], 5)
dataset["loan_intent"] = dataset["loan_intent"].replace(["MEDICAL"], 6)
dataset["loan_grade"] = dataset["loan_grade"].replace(["A"], 1)
dataset["loan_grade"] = dataset["loan_grade"].replace(["B"], 2)
dataset["loan_grade"] = dataset["loan_grade"].replace(["C"], 3)
dataset["loan_grade"] = dataset["loan_grade"].replace(["D"], 4)
dataset["loan_grade"] = dataset["loan_grade"].replace(["E"], 5)
dataset["loan_grade"] = dataset["loan_grade"].replace(["F"], 6)
dataset["loan_grade"] = dataset["loan_grade"].replace(["G"], 7)
# dataset = pd.get_dummies(dataset)
dataset

,Unnamed: 0,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,30786,41,40000,1,3.0,1,1,9200,7.49,0,0.23,0,13
1,29460,44,28000,2,0.0,2,1,3500,8.94,0,0.13,0,12
2,7059,22,56000,1,0.0,3,2,7000,11.36,0,0.13,0,2
3,5377,24,45000,3,2.0,1,1,7200,7.29,0,0.16,0,4
4,27170,28,55000,1,3.0,4,5,15000,17.06,0,0.27,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24430,9887,23,45000,1,7.0,5,2,9000,10.65,0,0.20,0,2
24431,15196,26,125000,3,3.0,5,4,24000,15.28,0,0.19,1,4
24432,32218,43,138000,3,4.0,5,1,9900,6.99,0,0.07,0,14
24433,25398,27,84780,3,5.0,2,1,18000,7.51,0,0.21,0,10


We then need to split the training and testing data from the dataset using sklearn

In [441]:
target_dataset = dataset['cb_person_default_on_file']
dataset.drop('cb_person_default_on_file', inplace=True, axis=1)
train_feature_dataset, test_feature_dataset, train_label_dataset, test_label_dataset  = train_test_split(dataset, target_dataset, test_size=0.2)

Run Naive Bayes, Decision Tree and KNN on the dataset

In [442]:
mnb = MultinomialNB()
gnb = GaussianNB()
predicted_dataset_mnb = mnb.fit(train_feature_dataset, train_label_dataset).predict(test_feature_dataset)
predicted_dataset_gnb = gnb.fit(train_feature_dataset, train_label_dataset).predict(test_feature_dataset)
print("MultinomialNB: Number of mislabeled points out of a total %d points : %d" % (test_feature_dataset.shape[0], (test_label_dataset != predicted_dataset_mnb).sum()))
print("MultinomialNB Accuracy:",metrics.accuracy_score(test_label_dataset, predicted_dataset_mnb), "\n")
print("GaussianNB: Number of mislabeled points out of a total %d points : %d" % (test_feature_dataset.shape[0], (test_label_dataset != predicted_dataset_gnb).sum()))
print("GaussianNB Accuracy:",metrics.accuracy_score(test_label_dataset, predicted_dataset_gnb), "\n")

MultinomialNB: Number of mislabeled points out of a total 4887 points : 2391
MultinomialNB Accuracy: 0.5107427869858809 

GaussianNB: Number of mislabeled points out of a total 4887 points : 862
GaussianNB Accuracy: 0.8236136689175363 



In [443]:
clf = DecisionTreeClassifier()
clf = clf.fit(train_feature_dataset, train_label_dataset)
# tree.plot_tree(clf)
predicted_dataset_dt = clf.predict(test_feature_dataset)
print("DecisionTreeClassifier: Number of mislabeled points out of a total %d points : %d" % (test_feature_dataset.shape[0], (test_label_dataset != predicted_dataset_dt).sum()))
print("DecisionTreeClassifier Accuracy:",metrics.accuracy_score(test_label_dataset, predicted_dataset_dt), "\n")

DecisionTreeClassifier: Number of mislabeled points out of a total 4887 points : 848
DecisionTreeClassifier Accuracy: 0.8264784121137713 



In [444]:
k = 2
neigh = KNeighborsClassifier(n_neighbors=k)
neigh = neigh.fit(train_feature_dataset, train_label_dataset)
predicted_dataset_knn = neigh.predict(test_feature_dataset)
print("KNeighborsClassifier: Number of mislabeled points out of a total %d points : %d" % (test_feature_dataset.shape[0], (test_label_dataset != predicted_dataset_knn).sum()))
print("KNeighborsClassifier Accuracy:",metrics.accuracy_score(test_label_dataset, predicted_dataset_knn), "\n")

KNeighborsClassifier: Number of mislabeled points out of a total 4887 points : 936
KNeighborsClassifier Accuracy: 0.8084714548802947 

